<a href="https://colab.research.google.com/github/mackly45/BDE-ESCIC/blob/main/model%20fraude%20bancaire.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Importation des bibliothèques nécessaires**

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, f1_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

 Monter Google ***Drive***

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Charger le fichier CSV**

In [3]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/transactions.csv')


Affichage des premières lignes du fichier pour vérifier la ***structure***

In [4]:
print("Aperçu des données :")
print(df.head())

Aperçu des données :
   transaction_id   amount                 time       location  account_age  \
0               1  3250.17  2024-07-11 00:00:00    Los Angeles            3   
1               2  4535.14  2024-07-11 01:00:00        Chicago            9   
2               3  1095.94  2024-07-11 02:00:00       New York            2   
3               4  3831.26  2024-07-11 03:00:00  San Francisco            3   
4               5  2742.76  2024-07-11 04:00:00        Houston            4   

  transaction_type  is_fraud  
0        transfert         1  
1         virement         0  
2            achat         0  
3         paiement         0  
4         virement         1  


## ***Partie 1 : Préparation des données***

Nettoyage des données : suppression des valeurs manquantes et des **doublons**

In [5]:
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)


Définition des colonnes catégorielles et **numériques**

In [6]:
categorical_cols = ['transaction_type', 'location']
numerical_cols = ['amount', 'account_age']


Séparation des variables d'entrée (X) et de sortie (y)

In [7]:
X = df.drop('is_fraud', axis=1)
y = df['is_fraud']

Division des données en ensembles d'entraînement et de test (80% / 20%)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


Création du préprocesseur pour les colonnes catégorielles et numériques

In [9]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ]
)

### ***Partie 2 : Entraînement des modèles***


Modèle de Régression Logistique

In [10]:
logistic_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression())
])
logistic_model.fit(X_train, y_train)
y_pred_logistic = logistic_model.predict(X_test)

Modèle de Forêt Aléatoire

In [11]:
forest_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier())
])
forest_model.fit(X_train, y_train)
y_pred_forest = forest_model.predict(X_test)

 Modèle de SVM (Support Vector Machine)**texte en gras**

In [12]:
svm_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', SVC())
])
svm_model.fit(X_train, y_train)
y_pred_svm = svm_model.predict(X_test)

### ***Partie 3 : Évaluation des modèles***

In [13]:
print("\nÉvaluation des modèles :")


Évaluation des modèles :


Précision, rappel, et F1-score pour chaque modèle

In [14]:
print("\nRégression Logistique :")
print(classification_report(y_test, y_pred_logistic))



Régression Logistique :
              precision    recall  f1-score   support

           0       0.40      0.24      0.30       102
           1       0.48      0.67      0.56       108

    accuracy                           0.46       210
   macro avg       0.44      0.45      0.43       210
weighted avg       0.44      0.46      0.43       210



In [15]:
print("\nForêt Aléatoire :")
print(classification_report(y_test, y_pred_forest))


Forêt Aléatoire :
              precision    recall  f1-score   support

           0       0.45      0.41      0.43       102
           1       0.48      0.52      0.50       108

    accuracy                           0.47       210
   macro avg       0.46      0.47      0.46       210
weighted avg       0.47      0.47      0.47       210



In [16]:
print("\nSVM :")
print(classification_report(y_test, y_pred_svm))


SVM :
              precision    recall  f1-score   support

           0       0.49      0.39      0.43       102
           1       0.52      0.61      0.56       108

    accuracy                           0.50       210
   macro avg       0.50      0.50      0.50       210
weighted avg       0.50      0.50      0.50       210



Comparaison des **modèles**

In [17]:
models = {
    "Régression Logistique": logistic_model,
    "Forêt Aléatoire": forest_model,
    "SVM": svm_model
}

# Choisir le modèle ayant les meilleures performances en fonction du F1-score

In [18]:
best_model = max(models, key=lambda model: f1_score(y_test, models[model].predict(X_test)))
print(f"\nLe meilleur modèle pour la détection des fraudes est : {best_model}")


Le meilleur modèle pour la détection des fraudes est : SVM
